In [2]:
import pandas as pd
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from custom_score.utils import cleanString
from datasets_loaders.loaders import load_billsum
from sklearn.cluster import SpectralClustering, Birch
from hdbscan import HDBSCAN

import matplotlib.pyplot as plt
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

### Dataset

In [ ]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

In [ ]:
subset = dataset.iloc[3:5, :]
subset

### ILP Tuning

In [3]:
sources = [
    "I work as a data scientist in an IT company. I love data science. I eat many fruits everyday.",
    "My neighbord often plays guitar. My mother is a very nice person. Sometimes I accompany him on the piano.",
    "Most days, I pratice sports. I run and go to the gym 5 times a week. It is complementary with intellectual activities."
]

golds = [
    "I am a pasionnate engineer.",
    "Music helps me connect with people.",
    "Physical activities occupy a consequent part of my life."
]

In [4]:
ms = MARSCore(sources, golds, precision_level="c", ratio=2, clusterizer=Birch(), printRange=(range(3)))
ms.compute()

In [5]:
_=ms.assess()

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Scores: 
   CBERT-R  CBERT-P  CBERT-F  BERTScore-R  BERTScore-P  BERTScore-F  BARTScore   R-1  R-2   R-L
0     0.72     0.47     0.57         0.86         0.91         0.88       0.13  0.25  0.0  0.25
1     0.18     0.28     0.22         0.88         0.86         0.87       0.21  0.00  0.0  0.00
2     0.31     0.29     0.30         0.87         0.86         0.87       0.34  0.09  0.0  0.09

Correlations: 
               pearson_CBERT_R-1  pearson_CBERT_R-2  pearson_CBERT_R-L  pearson_BERT_R-1  pearson_BERT_R-2  pearson_BERT_R-l  pearson_BART_R-1  pearson_BART_R-2  pearson_BART_R-l
Pearson score               0.99                NaN               0.99              0.93               NaN              0.93             -0.52               NaN             -0.52
p-value                     0.09                NaN               0.09              0.23               NaN              0.23              0.65               NaN              0.65


In [6]:
print(ms)

--------MARScore OBJECT--------

Number of Documents : 3
Corpus Avg Size     : 106
Refined Avg Size    : 31

-------------------------------

Corpus no.1 : 88.0%
I work as a data scientist in an IT company.
I love data science.
I eat many fruits everyday.

Corpus no.2 : 87.0%
My neighbord often plays guitar.
My mother is a very nice person.
Sometimes I accompany him on the piano.

Corpus no.3 : 87.0%
Most days, I pratice sports.
I run and go to the gym 5 times a week.
It is complementary with intellectual activities.

-------------------------------
